# Data Quality Analysis: NovaCred Credit Applications

**Dataset:** `raw_credit_applications.json`: 500+ loan application records  

## Scope

This notebook covers nine data quality issue categories:

| Section | Issue Category | Dimension |
|---------|---------------|-----------|
| 1 | Duplicate records | Accuracy |
| 2 | Missing / incomplete records | Completeness |
| 3 | Inconsistent data types | Consistency |
| 4 | Invalid or impossible values | Validity |
| 5 | Inconsistent categorical coding (gender) | Consistency |
| 6 | Inconsistent date formats (date of birth) | Consistency |
| 7 | Out-of-range numeric values | Validity |
| 8 | Nested spending_behavior field | Consistency |
| 9 | Redundant income field (annual_salary vs annual_income) | Consistency |

One structural fix is applied at load time before any section checks:
- **ZIP code** cast to string to prevent float coercion from the single missing value in that column

**How to read each section:** risk explanation → count of affected records (n and %) → remediation applied.

**How to use this notebook:** Run all cells top-to-bottom. `df_clean` carries all fixes forward and is saved to `data/cleaned_credit_applications.csv` at the end so the bias analysis and privacy notebooks can load it directly.

In [1]:
# Imports
import json
import re
import ast
from datetime import date
import pandas as pd

In [2]:
# Load JSON data
with open("../data/raw_credit_applications.json") as f:
    data = json.load(f)

# Normalize JSON into a flat table
df = pd.json_normalize(data)
n = len(df)

print(df.shape)
df.head(3)

(502, 21)


,_id,spending_behavior,processing_timestamp,applicant_info.full_name,applicant_info.email,applicant_info.ssn,applicant_info.ip_address,applicant_info.gender,applicant_info.date_of_birth,applicant_info.zip_code,...,financials.credit_history_months,financials.debt_to_income,financials.savings_balance,decision.loan_approved,decision.rejection_reason,loan_purpose,decision.interest_rate,decision.approved_amount,financials.annual_salary,notes
0,app_200,"[{'category': 'Shopping', 'amount': 480}, {'ca...",2024-01-15T00:00:00Z,Jerry Smith,jerry.smith17@hotmail.com,596-64-4340,192.168.48.155,Male,2001-03-09,10036,...,23,0.20,31212,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN
1,app_037,"[{'category': 'Rent', 'amount': 608}, {'catego...",NaN,Brandon Walker,brandon.walker2@yahoo.com,425-69-4784,10.1.102.112,M,1992-03-31,10032,...,51,0.18,17915,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN
2,app_215,"[{'category': 'Rent', 'amount': 109}]",NaN,Scott Moore,scott.moore94@mail.com,370-78-5178,10.240.193.250,Male,1989-10-24,10075,...,41,0.21,37909,True,NaN,vacation,3.7,59000.0,NaN,NaN


---
## 1. Duplicate Records

Risk: If the same application is entered more than once (due to errors, fraud, or resubmission), it can artificially increase the total number of applications and distort statistics such as approval rates. Duplicate applications can signal problems in the data processing pipeline or deliberate attempts to game the system.

We check two signals:
- Duplicate application IDs (_id): the primary key must be unique
- Duplicate name and email combinations: the same person applying more than once

In [3]:
# Check and report duplicate records by application ID and by name + email combination
dup_id    = df['_id'].duplicated(keep=False)
dup_combo = df.duplicated(subset=['applicant_info.full_name', 'applicant_info.email'], keep=False)

# Print count of duplicates for each type
print(f"Duplicate application IDs:      {dup_id.sum()} records  ({dup_id.sum()/len(df):.1%})")
print(f"Duplicate name + email:         {dup_combo.sum()} records  ({dup_combo.sum()/len(df):.1%})")

# Display duplicate records for manual review
if (dup_id | dup_combo).any():
    display(df[dup_id | dup_combo][['_id', 'applicant_info.full_name', 'applicant_info.email']])

Duplicate application IDs:      4 records  (0.8%)
Duplicate name + email:         4 records  (0.8%)


,_id,applicant_info.full_name,applicant_info.email
8,app_042,Joseph Lopez,joseph.lopez1@gmail.com
354,app_042,Joseph Lopez,joseph.lopez1@gmail.com
383,app_001,Stephanie Nguyen,stephanie.nguyen47@mail.com
455,app_001,Stephanie Nguyen,stephanie.nguyen47@mail.com


In [4]:
# Remove duplicate application IDs, keeping only the first occurrence
# .copy() ensures df_clean is an independent DataFrame (prevents SettingWithCopyWarning)
df_clean = df.drop_duplicates(subset='_id', keep='first').copy()
print(f"After deduplication: {len(df_clean)} records  (removed {n - len(df_clean)})")

After deduplication: 500 records  (removed 2)


---
## 2. Missing Values

**Risk:** Missing values in critical fields make records unusable for lending decisions and can introduce bias if missingness is non-random (e.g. if lower-income applicants disproportionately have income missing).

In [5]:
# Count missing values per column and sort by frequency
missing_count = df_clean.isnull().sum()
missing_pct   = missing_count / len(df_clean) * 100

missing_report = (
    pd.DataFrame({'missing_count': missing_count, 'missing_%': missing_pct.round(1)})
    .query('missing_count > 0')
    .sort_values('missing_%', ascending=False)
)
display(missing_report)

print("\nNote: financials.annual_salary shows 99% missing because only 5 records use it")
print("instead of annual_income; these two fields represent the same concept and are")
print("consolidated into annual_income in Section 9.")

,missing_count,missing_%
notes,500,100.0
financials.annual_salary,495,99.0
loan_purpose,450,90.0
processing_timestamp,438,87.6
decision.rejection_reason,292,58.4
decision.interest_rate,208,41.6
decision.approved_amount,208,41.6
financials.annual_income,5,1.0
applicant_info.ssn,4,0.8
applicant_info.ip_address,4,0.8



Note: financials.annual_salary shows 99% missing because only 5 records use it
instead of annual_income; these two fields represent the same concept and are
consolidated into annual_income in Section 9.


In [6]:
# Define fields required for a valid lending decision
critical_fields = [
    'applicant_info.full_name',
    'applicant_info.email',
    'financials.annual_income',
    'financials.credit_history_months',
]

# Flag records missing at least one critical field
df_clean.loc[:, 'has_critical_missing'] = df_clean[critical_fields].isnull().any(axis=1)
print(f"Records missing at least one critical field: {df_clean['has_critical_missing'].sum()}  ({df_clean['has_critical_missing'].mean():.1%})")

Records missing at least one critical field: 5  (1.0%)


---
## 3. Wrong Data Types

**Risk:** Financial fields stored as text (e.g. `"73000"` instead of `73000`) cannot be used in calculations. Coercing them to numbers may silently convert corrupt string entries into `NaN`, so we log what changes before and after.

In [7]:
# Define the financial columns that should be numeric
numeric_fields = [
    'financials.annual_income',
    'financials.credit_history_months',
    'financials.debt_to_income',
    'financials.savings_balance',
]

def is_non_numeric(val):
    return not isinstance(val, (int, float)) and pd.notna(val)

# Report non-numeric entries per field before coercion
print("Non-numeric entries per field (before coercion):")
for col in numeric_fields:
    non_numeric = df_clean[col].apply(is_non_numeric)
    label = col.split('.')[-1]
    print(f"  {label:<25}  {non_numeric.sum()} records  ({non_numeric.sum()/len(df_clean):.1%})")

Non-numeric entries per field (before coercion):
  annual_income              8 records  (1.6%)
  credit_history_months      0 records  (0.0%)
  debt_to_income             0 records  (0.0%)
  savings_balance            0 records  (0.0%)


In [8]:
# Force all financial fields to numeric; unparseable entries become NaN
for col in numeric_fields:
    df_clean.loc[:, col] = pd.to_numeric(df_clean[col], errors='coerce')

# Confirm data types after coercion
print("Data types confirmed after coercion:")
print(df_clean[numeric_fields].dtypes.to_string())

Data types confirmed after coercion:
financials.annual_income             object
financials.credit_history_months      int64
financials.debt_to_income           float64
financials.savings_balance            int64


---
## 4. Invalid Values

**Risk:** Values that are present but logically impossible (a negative credit history, a future date of birth, a garbled email) corrupt downstream analysis just as much as missing values. We detect and nullify them.

In [9]:
# Flag negative credit history (cannot have fewer than 0 months)
neg_credit = df_clean['financials.credit_history_months'] < 0

# Parse date of birth and flag values outside a plausible age range
dob         = pd.to_datetime(df_clean['applicant_info.date_of_birth'], errors='coerce')
today       = pd.Timestamp(date.today())
age         = (today - dob).dt.days / 365.25
future_dob  = dob > today
too_old     = age > 100
invalid_dob = future_dob | too_old

print(f"Negative credit history months:  {neg_credit.sum()}  ({neg_credit.sum()/len(df_clean):.1%})")
print(f"Future date of birth:            {future_dob.sum()}  ({future_dob.sum()/len(df_clean):.1%})")
print(f"Age > 100 years:                 {too_old.sum()}  ({too_old.sum()/len(df_clean):.1%})")

Negative credit history months:  2  (0.4%)
Future date of birth:            0  (0.0%)
Age > 100 years:                 0  (0.0%)


In [10]:
# Define email pattern and flag malformed addresses (must follow user@domain.tld)
EMAIL_RE = re.compile(r'^[\w\.\+\-]+@[\w\-]+\.[a-zA-Z]{2,}$')

def is_bad_email(val):
    if pd.isna(val):
        return False
    return not bool(EMAIL_RE.match(str(val)))

bad_email = df_clean['applicant_info.email'].apply(is_bad_email)
print(f"Malformed emails:  {bad_email.sum()}  ({bad_email.sum()/len(df_clean):.1%})")

Malformed emails:  11  (2.2%)


In [11]:
# Define SSN pattern and flag malformed entries (US format must be NNN-NN-NNNN)
SSN_RE = re.compile(r'^\d{3}-\d{2}-\d{4}$')

def is_bad_ssn(val):
    if pd.isna(val):
        return False
    return not bool(SSN_RE.match(str(val)))

bad_ssn = df_clean['applicant_info.ssn'].apply(is_bad_ssn)
print(f"Malformed SSNs:  {bad_ssn.sum()}  ({bad_ssn.sum()/len(df_clean):.1%})")

# Set impossible and malformed values to NaN
df_clean.loc[neg_credit,  'financials.credit_history_months'] = float('nan')
df_clean.loc[invalid_dob, 'applicant_info.date_of_birth']     = float('nan')
df_clean.loc[bad_email,   'applicant_info.email']             = float('nan')
df_clean.loc[bad_ssn,     'applicant_info.ssn']               = float('nan')
print("Impossible and malformed values set to NaN.")

Malformed SSNs:  0  (0.0%)
Impossible and malformed values set to NaN.


---
## 5. Inconsistent Categorical Values

**Risk:** The same gender can appear as `"Male"`, `"M"`, or `"male"`. Without standardisation, any group-level analysis (e.g. approval rates by gender) will split the same group into separate buckets, making results misleading.

In [12]:
# Show raw gender values before standardisation
print("Raw gender values in dataset:")
print(df_clean['applicant_info.gender'].value_counts(dropna=False).to_string())

Raw gender values in dataset:
applicant_info.gender
Male      194
Female    193
F          58
M          53
            2


In [13]:
# Map all spelling variants to a canonical two-value vocabulary
GENDER_MAP = {
    'Male':   'Male',   'male':   'Male',   'M': 'Male',   'm': 'Male',
    'Female': 'Female', 'female': 'Female', 'F': 'Female', 'f': 'Female',
}
df_clean.loc[:, 'applicant_info.gender_clean'] = df_clean['applicant_info.gender'].map(GENDER_MAP)

# Count variants that could not be mapped
unmapped = df_clean['applicant_info.gender_clean'].isna() & df_clean['applicant_info.gender'].notna()
print(f"Variants that could not be mapped to Male/Female: {unmapped.sum()}  ({unmapped.sum()/len(df_clean):.1%})")

# Show cleaned gender distribution
print("\nCleaned gender distribution:")
print(df_clean['applicant_info.gender_clean'].value_counts(dropna=False).to_string())

Variants that could not be mapped to Male/Female: 2  (0.4%)

Cleaned gender distribution:
applicant_info.gender_clean
Female    251
Male      247
NaN         2


---
## 6. Mixed Date Formats

**Risk:** The same field contains dates written in multiple formats (`YYYY-MM-DD`, `DD/MM/YYYY`, `MM/DD/YYYY`, etc.). Parsing without standardisation produces wrong ages or silently drops records.

In [14]:
# Define known date format patterns for recognition
DATE_FORMATS = {
    r'^\d{4}-\d{2}-\d{2}$':   'YYYY-MM-DD (ISO standard)',
    r'^\d{2}/\d{2}/\d{4}$':   'DD/MM/YYYY or MM/DD/YYYY',
    r'^\d{2}-\d{2}-\d{4}$':   'DD-MM-YYYY',
    r'^\d{2}\.\d{2}\.\d{4}$': 'DD.MM.YYYY',
    r'^\w+ \d{1,2},? \d{4}$': 'Month D YYYY (text)',
}

raw_dob   = df_clean['applicant_info.date_of_birth'].dropna().astype(str)
remaining = len(raw_dob)

# Count records matching each date format pattern; only print formats present in the data
print("Date format distribution:")
for pattern, label in DATE_FORMATS.items():
    count      = raw_dob.str.match(pattern).sum()
    remaining -= count
    if count > 0:
        print(f"  {label:<35}  {count:>4} records  ({count/len(df_clean):.1%})")
print(f"  {'Unrecognised':<35}  {remaining:>4} records  ({remaining/len(df_clean):.1%})")

Date format distribution:
  YYYY-MM-DD (ISO standard)             339 records  (67.8%)
  DD/MM/YYYY or MM/DD/YYYY              101 records  (20.2%)
  Unrecognised                           60 records  (12.0%)


In [15]:
# Parse all date formats into a single consistent datetime column
df_clean.loc[:, 'applicant_info.date_of_birth_parsed'] = pd.to_datetime(
    df_clean['applicant_info.date_of_birth'], errors='coerce'
)

# Report how many dates remain unparseable
still_null = df_clean['applicant_info.date_of_birth_parsed'].isna().sum()
print(f"Records where date still could not be parsed: {still_null}  ({still_null/len(df_clean):.1%})")

Records where date still could not be parsed: 161  (32.2%)


---
## 7. Out-of-Range Numeric Values

**Risk:** Values that are numeric and parseable but structurally impossible (a debt-to-income ratio above 1, a negative savings balance, or zero/negative income) corrupt averages and any model trained on the data. We also flag extreme statistical outliers (more than 3 standard deviations from the mean income) for analyst review. Unlike structurally impossible values, outliers are *not* automatically removed. They are retained in `df_clean` to inspect before modelling.

In [16]:
def is_invalid_dti(val):
    return pd.notna(val) and not (0 <= val <= 1)

# Flag debt-to-income ratios outside the valid range [0, 1]
dti_out = df_clean['financials.debt_to_income'].apply(is_invalid_dti)

# Flag negative savings balances and zero or negative income
neg_savings = df_clean['financials.savings_balance'] < 0
neg_income  = df_clean['financials.annual_income']   <= 0

print(f"Debt-to-income ratio outside [0, 1]:  {dti_out.sum()}  ({dti_out.sum()/len(df_clean):.1%})")
print(f"Negative savings balance:             {neg_savings.sum()}  ({neg_savings.sum()/len(df_clean):.1%})")
print(f"Zero or negative annual income:       {neg_income.sum()}  ({neg_income.sum()/len(df_clean):.1%})")

Debt-to-income ratio outside [0, 1]:  1  (0.2%)
Negative savings balance:             1  (0.2%)
Zero or negative annual income:       1  (0.2%)


In [17]:
# Flag income statistical outliers: more than 3 standard deviations from the mean
income_mean     = df_clean['financials.annual_income'].mean()
income_std      = df_clean['financials.annual_income'].std()
income_outliers = (df_clean['financials.annual_income'] - income_mean).abs() > 3 * income_std

print(f"Income statistical outliers (> 3σ):   {income_outliers.sum()}  ({income_outliers.sum()/len(df_clean):.1%})")
print(f"Income range: ${df_clean['financials.annual_income'].min():,.0f} to ${df_clean['financials.annual_income'].max():,.0f}")

Income statistical outliers (> 3σ):   2  (0.4%)
Income range: $0 to $171,000


In [18]:
# Set structurally impossible values to NaN; statistical outliers are flagged but kept
df_clean.loc[dti_out,    'financials.debt_to_income']  = float('nan')
df_clean.loc[neg_savings,'financials.savings_balance'] = float('nan')
df_clean.loc[neg_income, 'financials.annual_income']   = float('nan')
print("Impossible values set to NaN. Outliers retained for analyst review before modelling.")

Impossible values set to NaN. Outliers retained for analyst review before modelling.


---
## 8. Nested `spending_behavior`: Flattening to Wide Format

**Risk:** The `spending_behavior` field arrives from JSON as a list of `{category, amount}` objects. Keeping it as a raw list makes the column unreadable in CSV and unusable for any numerical analysis. We pivot each unique category into its own numeric column (`spending_<Category>`); applicants without a given category receive `NaN`.

In [19]:
def parse_spending(val):
    # Accept either a Python list (direct from JSON) or a repr string (reloaded from CSV)
    if isinstance(val, list):
        return val
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except Exception:
            return []
    return []

def entries_to_dict(entries):
    # Convert a list of {category, amount} objects into a flat {spending_Category: amount} dict
    return {f"spending_{e['category']}": e['amount'] for e in entries}

In [20]:
# Parse the spending list and pivot each category into its own column
spending_wide = (
    df_clean['spending_behavior']
    .apply(parse_spending)
    .apply(entries_to_dict)
    .apply(pd.Series)
)

categories = sorted(c.replace('spending_', '') for c in spending_wide.columns)
print(f"Spending categories found ({len(categories)}): {categories}")
print(f"\nRecords with at least one spending entry: "
      f"{spending_wide.notna().any(axis=1).sum()}  "
      f"({spending_wide.notna().any(axis=1).sum()/len(df_clean):.1%})")

Spending categories found (15): ['Adult Entertainment', 'Alcohol', 'Dining', 'Education', 'Entertainment', 'Fitness', 'Gambling', 'Groceries', 'Healthcare', 'Insurance', 'Rent', 'Shopping', 'Transportation', 'Travel', 'Utilities']

Records with at least one spending entry: 500  (100.0%)


In [21]:
# Replace the raw nested column with flat numeric columns
df_clean = df_clean.drop(columns=['spending_behavior']).join(spending_wide)
print(f"df_clean shape after flattening spending_behavior: {df_clean.shape}")

df_clean shape after flattening spending_behavior: (500, 38)


---
## 9. Redundant Income Field: annual_salary vs annual_income

**Dimension:** Consistency  
**Risk:** Five records populate `annual_salary` instead of `annual_income` to record the applicant's income. Both columns represent the same concept under two different names. Aggregations on `annual_income` alone silently exclude these five records, biasing any income-based statistics or model features.

In [22]:
# Identify records where annual_salary is filled and annual_income is null
salary_mask = df_clean['financials.annual_salary'].notna()

print(f"Records with annual_salary present:           {salary_mask.sum()}")
print(f"Of those, annual_income is also null:         {(salary_mask & df_clean['financials.annual_income'].isna()).sum()}")
print()
display(df_clean.loc[salary_mask, ['_id', 'financials.annual_income', 'financials.annual_salary']])

Records with annual_salary present:           5
Of those, annual_income is also null:         5



,_id,financials.annual_income,financials.annual_salary
76,app_436,NaN,45000.0
94,app_421,NaN,46000.0
99,app_479,NaN,94000.0
141,app_463,NaN,86000.0
149,app_449,NaN,75000.0


In [23]:
# Copy annual_salary into annual_income where income is missing
df_clean.loc[salary_mask, 'financials.annual_income'] = df_clean.loc[salary_mask, 'financials.annual_salary']

# Drop the redundant annual_salary column
df_clean = df_clean.drop(columns=['financials.annual_salary'])

print(f"annual_salary values merged into annual_income. Column dropped.")
print(f"df_clean shape: {df_clean.shape}")

annual_salary values merged into annual_income. Column dropped.
df_clean shape: (500, 37)


In [24]:
# Export the cleaned dataset for use in downstream notebooks
out_path = "../data/cleaned_credit_applications.csv"
df_clean.to_csv(out_path, index=False)

print(f"Saved: {out_path}")
print(f"Shape: {df_clean.shape[0]} rows x {df_clean.shape[1]} columns")
print()
print("Load in downstream notebooks with:")
print("  import pandas as pd")
print("  df = pd.read_csv('../data/cleaned_credit_applications.csv',")
print("                   dtype={'applicant_info.zip_code': str})")

Saved: ../data/cleaned_credit_applications.csv
Shape: 500 rows x 37 columns

Load in downstream notebooks with:
  import pandas as pd
  df = pd.read_csv('../data/cleaned_credit_applications.csv',
                   dtype={'applicant_info.zip_code': str})
